In [11]:
import os

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa
import src.model as models
from sklearn.metrics import auc, confusion_matrix, roc_curve
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.metrics import AUC, BinaryAccuracy, Precision, Recall
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import src.graphics as graphics

In [12]:
VERSION = "3"
MODEL_BASE_NAME = "MegaClassifier_a_MobileNetV2"

DATASET_CSV = os.path.abspath(
    "./data/processed/onlyDetectionsForTrain/onlyDetectionsForTrain.csv"
)
DATASET_PATH = os.path.dirname(DATASET_CSV)

In [13]:
dataset = pd.read_csv(DATASET_CSV, sep=";")
dataset["file_name"] = dataset["file_name"].apply(
    lambda x: os.path.join(DATASET_PATH, x)
)
dataset["binary_label"] = dataset["binary_label"].astype(str)

train_dataset = dataset[dataset["subset"] == "train"]
validationtrain_dataset = dataset[dataset["subset"] == "validation"]
test_dataset = dataset[dataset["subset"] == "test"]

EPOCHS = 10
IMAGE_SIZE = (224, 224)
IMAGE_SHAPE = IMAGE_SIZE + (3,)
SEED = 42

In [14]:
OPTIMIZERS = {
    "Adam": tf.keras.optimizers.legacy.Adam(),
}

In [15]:
LOSS = {
    "BinaryFocalCrossentropy": lambda x, y: tf.keras.losses.BinaryFocalCrossentropy(
        alpha=x, gamma=y
    ),
}

ALPHAS = [0.25, 0.4, 0.5, 0.6, 0.75]

GAMMAS = [1, 2, 3, 4, 5]

In [16]:
METRICS = [
    BinaryAccuracy(name="accuracy"),
    Precision(name="precision"),
    Recall(name="recall"),
    AUC(name="auc"),
]

In [17]:
BATCH_SIZE = 64

In [ ]:
SUBVERSION = 0
SUB_SUBVERSION = 0

for ALPHA in ALPHAS:
    for GAMMA in GAMMAS:
        LOGS_PATH = os.path.abspath(
            f"./logs/{MODEL_BASE_NAME}/v{VERSION}/v{VERSION}.{SUBVERSION}.{SUB_SUBVERSION}"
        )
        MODELS_PATH = os.path.abspath(
            f"./models/{MODEL_BASE_NAME}/v{VERSION}/v{VERSION}.{SUBVERSION}.{SUB_SUBVERSION}"
        )
        REPORTS_PATH = os.path.abspath(
            f"./reports/{MODEL_BASE_NAME}/v{VERSION}/v{VERSION}.{SUBVERSION}.{SUB_SUBVERSION}"
        )

        MODEL_COMPLETE_NAME = (
            f"{MODEL_BASE_NAME} v{VERSION}.{SUBVERSION}.{SUB_SUBVERSION}"
        )

        train_datagen = ImageDataGenerator(
            preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
        )
        train_images = train_datagen.flow_from_dataframe(
            dataframe=train_dataset,
            x_col="file_name",
            y_col="binary_label",
            target_size=IMAGE_SIZE,
            batch_size=BATCH_SIZE,
            class_mode="binary",
            shuffle=True,
            seed=SEED,
        )

        datagen = ImageDataGenerator(
            preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
        )
        validation_images = datagen.flow_from_dataframe(
            dataframe=validationtrain_dataset,
            x_col="file_name",
            y_col="binary_label",
            target_size=IMAGE_SIZE,
            batch_size=BATCH_SIZE,
            class_mode="binary",
            shuffle=True,
            seed=SEED,
        )
        test_images = datagen.flow_from_dataframe(
            dataframe=test_dataset,
            x_col="file_name",
            y_col="binary_label",
            target_size=IMAGE_SIZE,
            batch_size=BATCH_SIZE,
            class_mode="binary",
            shuffle=False,
            seed=SEED,
        )

        pretrained_model = tf.keras.applications.MobileNetV2(
            weights="imagenet",
            include_top=False,
            input_shape=IMAGE_SHAPE,
        )
        pretrained_model.trainable = False

        model = tf.keras.Sequential(
            [
                pretrained_model,
                tf.keras.layers.GlobalAveragePooling2D(),
                tf.keras.layers.Dense(1, activation="sigmoid"),
            ],
            name=f"{MODEL_BASE_NAME}_v{VERSION}.{SUBVERSION}.{SUB_SUBVERSION}",
        )

        model.compile(
            optimizer=OPTIMIZERS["Adam"],
            loss=LOSS["BinaryFocalCrossentropy"](ALPHA, GAMMA),
            metrics=METRICS,
        )

        history = model.fit(
            train_images,
            epochs=EPOCHS,
            validation_data=validation_images,
            callbacks=[TensorBoard(log_dir=LOGS_PATH)],
        )

        dataframe = pd.DataFrame(history.history)
        history_path = os.path.join(
            LOGS_PATH, f"history_v{VERSION}.{SUBVERSION}.{SUB_SUBVERSION}.csv"
        )
        dataframe.to_csv(history_path, sep=";", index=False)

        os.makedirs(REPORTS_PATH, exist_ok=True)
        accuracy_chart = graphics.create_training_accuracy_chart(
            history_path=history_path,
            model_name=MODEL_COMPLETE_NAME,
        )
        accuracy_chart.write_image(
            f"{REPORTS_PATH}/accuracy_v{VERSION}.{SUBVERSION}.{SUB_SUBVERSION}.png"
        )

        loss_chart = graphics.create_training_loss_chart(
            history_path=history_path,
            model_name=MODEL_COMPLETE_NAME,
        )
        loss_chart.write_image(
            f"{REPORTS_PATH}/loss_v{VERSION}.{SUBVERSION}.{SUB_SUBVERSION}.png"
        )

        os.makedirs(MODELS_PATH, exist_ok=True)
        model.save(MODELS_PATH)

        results = model.evaluate(test_images)
        metric_names = history.model.metrics_names
        evaluation_results = {
            ("test_" + name): value for name, value in zip(metric_names, results)
        }

        evaluation = pd.DataFrame([evaluation_results])
        evaluation.to_csv(
            os.path.join(
                LOGS_PATH, f"evaluation_v{VERSION}.{SUBVERSION}.{SUB_SUBVERSION}.csv"
            ),
            sep=";",
            index=False,
        )

        y_pred_prob = model.predict(test_images)
        y_true = test_images.labels

        fpr, tpr, thresholds = roc_curve(y_true, y_pred_prob)
        roc_auc = auc(fpr, tpr)

        roc_curve_chart = graphics.create_roc_curve_chart(
            fpr=fpr,
            tpr=tpr,
            roc_auc=roc_auc,
            model_name=MODEL_COMPLETE_NAME,
        )
        roc_curve_chart.write_image(
            f"{REPORTS_PATH}/roc_curve_v{VERSION}.{SUBVERSION}.{SUB_SUBVERSION}.png"
        )

        optimal_idx = np.argmax(tpr - fpr)
        optimal_threshold = thresholds[optimal_idx]

        y_pred_class = (y_pred_prob > optimal_threshold).astype(int)
        conf_matrix = confusion_matrix(y_true, y_pred_class)
        conf_matrix_text = [[str(value) for value in row] for row in conf_matrix]

        confusion_matrix_chart = graphics.create_confusion_matrix_chart(
            conf_matrix=conf_matrix,
            conf_matrix_text=conf_matrix_text,
            model_name=MODEL_COMPLETE_NAME,
        )
        confusion_matrix_chart.write_image(
            f"{REPORTS_PATH}/confusion_matrix_optimal_v{VERSION}.{SUBVERSION}.{SUB_SUBVERSION}.png"
        )

        y_pred_class = (y_pred_prob > 0.5).astype(int)
        conf_matrix = confusion_matrix(y_true, y_pred_class)
        conf_matrix_text = [[str(value) for value in row] for row in conf_matrix]

        confusion_matrix_chart = graphics.create_confusion_matrix_chart(
            conf_matrix=conf_matrix,
            conf_matrix_text=conf_matrix_text,
            model_name=MODEL_COMPLETE_NAME,
        )
        confusion_matrix_chart.write_image(
            f"{REPORTS_PATH}/confusion_matrix_v{VERSION}.{SUBVERSION}.{SUB_SUBVERSION}.png"
        )

        SUB_SUBVERSION += 1

    SUBVERSION += 1
    SUB_SUBVERSION = 0
    print("\n\n")


Found 17906 validated image filenames belonging to 2 classes.
Found 4286 validated image filenames belonging to 2 classes.
Found 4286 validated image filenames belonging to 2 classes.
Epoch 1/10


2025-03-01 18:08:21.739910: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


280/280 [==============================] - ETA: 0s - loss: 0.1550 - accuracy: 0.8644 - precision: 0.8962 - recall: 0.9234 - auc: 0.9225

2025-03-01 18:09:10.535118: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


280/280 [==============================] - 61s 211ms/step - loss: 0.1550 - accuracy: 0.8644 - precision: 0.8962 - recall: 0.9234 - auc: 0.9225 - val_loss: 0.1072 - val_accuracy: 0.9118 - val_precision: 0.9315 - val_recall: 0.9355 - val_auc: 0.9707
Epoch 2/10
280/280 [==============================] - 57s 205ms/step - loss: 0.1056 - accuracy: 0.9122 - precision: 0.9368 - recall: 0.9449 - auc: 0.9657 - val_loss: 0.0967 - val_accuracy: 0.9235 - val_precision: 0.9406 - val_recall: 0.9439 - val_auc: 0.9759
Epoch 3/10
280/280 [==============================] - 58s 206ms/step - loss: 0.0955 - accuracy: 0.9200 - precision: 0.9438 - recall: 0.9482 - auc: 0.9722 - val_loss: 0.0966 - val_accuracy: 0.9230 - val_precision: 0.9563 - val_recall: 0.9260 - val_auc: 0.9767
Epoch 4/10
280/280 [==============================] - 60s 213ms/step - loss: 0.0913 - accuracy: 0.9253 - precision: 0.9476 - recall: 0.9515 - auc: 0.9746 - val_loss: 0.0949 - val_accuracy: 0.9249 - val_precision: 0.9476 - val_recall: 

...

280/280 [==============================] - 60s 211ms/step - loss: 0.0177 - accuracy: 0.8822 - precision: 0.9219 - recall: 0.9149 - auc: 0.9416 - val_loss: 0.0116 - val_accuracy: 0.8957 - val_precision: 0.8854 - val_recall: 0.9676 - val_auc: 0.9658
Epoch 2/10
280/280 [==============================] - 58s 206ms/step - loss: 0.0087 - accuracy: 0.9017 - precision: 0.9327 - recall: 0.9344 - auc: 0.9604 - val_loss: 0.0084 - val_accuracy: 0.9127 - val_precision: 0.9563 - val_recall: 0.9097 - val_auc: 0.9704
Epoch 3/10
280/280 [==============================] - 57s 204ms/step - loss: 0.0073 - accuracy: 0.9120 - precision: 0.9396 - recall: 0.9415 - auc: 0.9684 - val_loss: 0.0077 - val_accuracy: 0.9176 - val_precision: 0.9271 - val_recall: 0.9503 - val_auc: 0.9737
Epoch 4/10
280/280 [==============================] - 57s 205ms/step - loss: 0.0067 - accuracy: 0.9194 - precision: 0.9443 - recall: 0.9468 - auc: 0.9729 - val_loss: 0.0074 - val_accuracy: 0.9172 - val_precision: 0.9606 - val_recall: 

INFO:tensorflow:Assets written to: /Users/litto/WORKSPACE/MegaClassifier/models/MegaClassifier_a_MobileNetV2/v3/v3.4.24/assets


67/67 [==============================] - 12s 169ms/step - loss: 0.0070 - accuracy: 0.9260 - precision: 0.9458 - recall: 0.9422 - auc: 0.9782


2025-03-01 22:22:23.589926: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


67/67 [==============================] - 11s 161ms/step



